In [1]:
from py.tap_trial import tap_trial

import numpy as np
import pandas as pd

net_filename = 'SiouxFalls_net.tntp'
true_flows = pd.read_pickle('py/gravity_model/true_flows.pkl')['flow']
true_flows.head()

link
(1,2)    4494.611949
(1,3)    8119.023795
(2,1)    4519.023795
(2,6)    5967.349609
(3,1)    8094.611949
Name: flow, dtype: float64

# Optimizing inputs to a gravity model used to construct and OD matrix

Compared against "ground truth" flows from a reference OD matrix.

## Verifying the cost function

RMSE against reference true flows.

In [2]:
%%time
rmse = tap_trial(netFileName=net_filename, true_flows=true_flows, params=np.ones(276))#np.arange(1,277))
rmse

CPU times: user 15.6 ms, sys: 15.6 ms, total: 31.2 ms
Wall time: 59.9 ms


4307.949280090229

# Trying different optimization algorithms

## Hill climbing algorithm

In [3]:
class HillClimbing:
    
    def __init__(self, n=276, sigma=1, starter_params=None):        
        self.n = n
        self.sigma = 1 # sd of lognormal random components
        self.i = 0 # tracks how many iterations we've run so far
        self.best = starter_params if starter_params else self.random_params()
        self.minscore = tap_trial(netFileName=net_filename, true_flows=true_flows, params=self.best)
        print(f'0:\t\t {self.minscore}')
    
    def random_params(self):
        return np.random.lognormal(0, self.sigma, self.n)

    def mutate_params(self, params_to_change):
        new_params = params_to_change.copy()
        index_to_mutate = np.random.randint(self.n)
        new_params[index_to_mutate] = np.random.lognormal(np.log(new_params[index_to_mutate]), self.sigma, 1)
        return new_params
    
    def iterate_solutions(self, max_steps, score_cutoff=0, save_progress=False):
        
        if save_progress and not self.steps_progress:
            self.steps_progress = []
            self.score_progress = []
        
        while self.minscore > score_cutoff:
            if self.i >= max_steps:
                break
            self.i += 1
            
            next_params = self.mutate_params(self.best)
            next_score = tap_trial(netFileName=net_filename, true_flows=true_flows, params=next_params)
            
            if next_score < self.minscore:
                self.best = next_params
                self.minscore = next_score
                if self.i%100==0:
                    print(f'{self.i}:\t\t {self.minscore}')
                if save_progress:
                    self.steps_progress.append(self.i)
                    self.score_progress.append(self.minscore)
        
        if save_progress:
            return self.best, self.minscore, self.steps_progress, self.score_progress
        return self.best, self.minscore

hc = HillClimbing()
hc_best, hc_score = hc.iterate_solutions(max_steps=5_000)
hc_score

0:		 3896.4398771874967
100:		 3603.5556914673825
800:		 1376.1213114466186
1500:		 1047.2193177563854
1700:		 1020.286254659699
1800:		 1016.4191818505192
2400:		 1000.7462525174379
3400:		 995.6856737399592
4000:		 994.8736882582709
4200:		 994.0682614399138


993.0392479752553

In [4]:
hc_score

993.0392479752553

## Simulated Annealing

In [5]:
class SimulatedAnnealing(HillClimbing):
    
    def iterate_solutions(self, max_steps, score_cutoff=0, save_progress=False, T=1.0, T_min=0.00001, alpha=0.9, iters_per_temp=100):
        # T_min is not used

        if save_progress and not self.steps_progress:
            self.steps_progress = []
            self.score_progress = []

        current_params = self.best
        current_score = self.minscore
        
        while T>T_min:
            for j in range(iters_per_temp):
                if self.i >= max_steps or self.minscore <= score_cutoff:
                    break
                self.i += 1
                
                next_params = self.mutate_params(current_params)
                next_score = tap_trial(netFileName=net_filename, true_flows=true_flows, params=next_params)                
                
                if (next_score < current_score) or (
                        (next_score-current_score<100) and
                        (1/(1+np.exp((next_score-current_score)/T)))):
                    current_params = next_params
                    current_score = next_score

                    if next_score < self.minscore:
                        self.best = next_params
                        self.minscore = next_score
                        if self.i%10==0:
                            print(f'{self.i}:\t\t {self.minscore}')
                        if save_progress:
                            self.steps_progress.append(self.i)
                            self.score_progress.append(self.minscore)
                        # print(next_score)

            T *= alpha

        if save_progress:
            return self.best, self.minscore, self.steps_progress, self.score_progress
        return self.best, self.minscore

sa = SimulatedAnnealing()
sa_best, sa_score = sa.iterate_solutions(max_steps=5_000)
sa_score


0:		 4202.765054940466
60:		 4115.897996264641
70:		 3987.9430406105043
110:		 3743.036128982929
150:		 3566.2870408958615
260:		 3449.1742716781305
270:		 3278.6650388506855
410:		 3081.5017264673056
500:		 3008.1231331304
760:		 2338.8909251853725
1150:		 2221.6667031957018
1370:		 1866.1177105995362
<ipython-input-5-ebbbdf8e2c23>:24: RuntimeWarning: overflow encountered in exp
  (1/(1+np.exp((next_score-current_score)/T)))):
3320:		 1474.8864053593368
3380:		 1384.5047326644321
3390:		 1381.6851769188106
3670:		 1245.5070243332066
3680:		 1225.519514793397
3700:		 1219.6896972541674
3740:		 1185.2494867786077
3750:		 1184.326549771466
3810:		 1026.0620443080984
3850:		 992.3580489772754
3950:		 869.2337596869045
3970:		 861.2761412636547
4010:		 842.8315738006341
4040:		 790.318394466545
4050:		 747.1272324786898
4150:		 682.7431723658694
4170:		 675.1222941953797
4280:		 630.2201996116509
4290:		 628.9413819906375
4420:		 609.5832895308608
4590:		 576.565559169877
4660:		 545.50175

496.1600564129279